# Airnow Highest 5 Uploader

Uploads data for Airnow's highest 5 AQI locations to ESDR.

Reports to stat.createlab.org as `Airnow Highest Five - Uploader`.

Airnow's docs for the highest 5 are here: https://airnow.gov/index.cfm?action=airnow.news_item&newsitemid=103

In [0]:
import json, os, dateutil, re, requests, subprocess, datetime, glob, stat, urllib.parse

from dateutil import rrule, tz, parser
from sqlitedict import SqliteDict

In [0]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb


def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


exec_ipynb('./python-utils/utils.ipynb')
exec_ipynb('./python-utils/esdr-library.ipynb')
exec_ipynb('./airnow-common.ipynb')

In [0]:
STAT_SERVICE_NAME = 'Airnow Highest Five - Uploader'
STAT_HOSTNAME = 'hal21'
STAT_SHORTNAME = 'airnow-highest-five-uploader'

NUM_FILES_PER_UPLOAD_BATCH = 500

RUN_INTERVAL_SECONDS = 60 * 5   # every 5 minutes

# This file stores the Geocoding API key named 'airnow-highest-5-uploader.ipynb (hal21)', defined under
# the 'Hal21 Cocalc Notebooks' project in the lab admin Google account at https://console.developers.google.com/
GOOGLE_API_KEYS_JSON = './google-api-keys.json'

# Load the Google API key
google_api_keys = {}
with open(GOOGLE_API_KEYS_JSON, 'r') as f:
    google_api_keys = json.load(f)

GEOCODING_API_KEY = google_api_keys['geocoding']
GEOCODING_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json?key='+GEOCODING_API_KEY+'&address='

In [0]:
Stat.set_service(STAT_SERVICE_NAME)

In [0]:
uploaded_file_timestamps_db = SqliteDict(AirnowCommon.HIGHEST_FIVE_AQI_DIRECTORY + '/uploaded_file_timestamps.db', autocommit=True)

In [0]:
accumulated_cities = {}
accumulated_rankings = {}
accumulated_file_timestamps = {}

def clear_accumulated():
    global accumulated_cities, accumulated_rankings, accumulated_file_timestamps
    accumulated_cities = {}
    accumulated_rankings = {}
    accumulated_file_timestamps = {}

# Record format example: 1583863202.348545:1,235,111|2,809,93|3,789,91|4,946,86|5,230,81
# A colon separates the Unix timestamp from the rankings.  Rankings are pipe delimited and there should exist 5 per timestamp.
# A ranking item consists of three comma-delimited values: the rank index [1-5], the Airnow city ID, and the AQI
def process_dat_file(src):
    src_epoch_timestamp = os.path.getmtime(src)
    dt = datetime.datetime.strptime(os.path.basename(src), '%Y%m%d.dat')
    epoch_time = (dt - datetime.datetime(1970, 1, 1)).total_seconds()
    Stat.debug('Processing file %s' % src, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

    num_records_read = 0
    with open(src, 'r') as records:
        lineno = 0
        error_count = 0
        for record in records:
            lineno += 1
            try:
                (timestamp, rankings) = record.split(':')
                timestamp = float(timestamp)
                for ranking in rankings.split('|'):
                    (rank, city_id, aqi) = map(int,ranking.split(','))

                    if city_id not in accumulated_cities:
                        accumulated_cities[city_id] = []
                    accumulated_cities[city_id].append([timestamp, rank, aqi])

                    if rank not in accumulated_rankings:
                        accumulated_rankings[rank] = []
                    accumulated_rankings[rank].append([timestamp, city_id, aqi])

            except:
                Stat.warning('Failed to parse line %d of %s. Skipping.' % (lineno, src), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
                error_count += 1
                continue

            num_records_read += 1

        if error_count > 5:
            raise Exception('Too many parse errors (%d) reading %s, aborting' % (error_count, src))

    if error_count > 0:
        Stat.warning('Read %d records from %s (%d error(s))' % (num_records_read, src, error_count), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    else:
        Stat.debug('Read %d records from %s (%d error(s))' % (num_records_read, src, error_count), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    accumulated_file_timestamps[src] = src_epoch_timestamp

# process_dat_file('../../airnow-data/highest-five-aqi/dat/20200310.dat')
# print(json.dumps(accumulated_rankings, sort_keys=True, indent=3))
# print(json.dumps(accumulated_cities, sort_keys=True, indent=3))

In [0]:
cities_cached = None

def refresh_city_info_cache():
    global cities_cached
    with open(AirnowCommon.HIGHEST_FIVE_AQI_DIRECTORY + '/airnow_city_id_to_city_info.json', 'r') as f:
        cities_cached = json.load(f)

def get_city_info(city_id):
    global cities_cached
    if not cities_cached:
        refresh_city_info_cache();

    try:
        return cities_cached[str(city_id)]
    except:
        return None

# print(json.dumps(get_city_info(164), sort_keys=True, indent=3))  # {"city": "Pittsburgh", "state": "PA"}
# print(json.dumps(get_city_info('164'), sort_keys=True, indent=3))  # {"city": "Pittsburgh", "state": "PA"}
# print(json.dumps(get_city_info('-1'), sort_keys=True, indent=3))  # null

In [0]:
esdr = None
highest_five_city_esdr_product = None
highest_five_ranking_esdr_product = None

In [0]:
def get_highest_five_city_esdr_product():
    global esdr, highest_five_city_esdr_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not highest_five_city_esdr_product:
        highest_five_city_esdr_product = esdr.get_product_by_name('airnow_aqi_highest_five_city')
    return highest_five_city_esdr_product

def get_highest_five_ranking_esdr_product():
    global esdr, highest_five_ranking_esdr_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not highest_five_ranking_esdr_product:
        highest_five_ranking_esdr_product = esdr.get_product_by_name('airnow_aqi_highest_five_ranking')
    return highest_five_ranking_esdr_product

# print(json.dumps(get_highest_five_city_esdr_product(), sort_keys=True, indent=3))  # null
# print(json.dumps(get_highest_five_ranking_esdr_product(), sort_keys=True, indent=3))  # null

In [0]:
# docs at https://developers.google.com/maps/documentation/geocoding/intro
def get_lat_lon_for_address(city, state):
    try:
        address = "%s, %s" % (city, state)
        response = requests.get(GEOCODING_API_URL + urllib.parse.quote(address))
        if (response.status_code >= 200 and response.status_code < 300):
            geocode_results = response.json()
            if geocode_results and \
                geocode_results['results'] and \
                geocode_results['results'][0] and \
                geocode_results['results'][0]['geometry'] and \
                geocode_results['results'][0]['geometry']['location'] and \
                geocode_results['results'][0]['geometry']['location']['lat'] and \
                geocode_results['results'][0]['geometry']['location']['lng']:
                location = geocode_results['results'][0]['geometry']['location']
                return {
                    "lat" : location['lat'],
                    "lon" : location['lng']
                }
            else:
                Stat.warning("Failed to get geocode results for address [%s]" % (address, response.status_code), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        else:
            Stat.warning("Failed to geocode address [%s] (HTTP %d)" % (address, response.status_code), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    except requests.HTTPError as e:
        Stat.warning("Failed to geocode address [%s] (HTTP %d)" % (address, e.response.status_code), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    except:
        Stat.warning("Failed to geocode address [%s]" % (address), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    return None

# get_lat_lon_for_address('Pittsburgh','PA')   # {'lat': 40.44062479999999, 'lon': -79.9958864}

In [0]:
def upload_city(city_id):
    global esdr, highest_five_city_esdr_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not highest_five_city_esdr_product:
        highest_five_city_esdr_product = get_highest_five_city_esdr_product()

    city_info = get_city_info(city_id)

    if city_info:
        print("Uploading city id [%d]" % city_id)

        city_and_state = "%s, %s [%d]" % (city_info['city'], city_info['state'], city_id)
        device = esdr.get_or_create_device(highest_five_city_esdr_product, serial_number=str(city_id), name=city_and_state)

        if device:
            feed = esdr.get_feed(device)
            if feed == None:
                # attempt to geocode
                lat_lon = get_lat_lon_for_address(city_info['city'], city_info['state'])
                lat = lat_lon['lat'] if lat_lon else None
                lon = lat_lon['lon'] if lat_lon else None

                # create the feed
                feed = esdr.get_or_create_feed(device, lat=lat, lon=lon)
                Stat.info('ESDR feed created for city id %d' % (city_id), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

            if feed:
                if city_id in accumulated_cities:
                    records = accumulated_cities[city_id]

                    try:
                        esdr.upload(feed, {
                            'channel_names': ['rank', 'aqi'],
                            'data': records
                        })
                        Stat.info('%s: Uploaded %d records' % (device['name'], len(records)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
                    except requests.HTTPError as e:
                        Stat.warning('%s: Failed to upload %d records (HTTP %d)' % (device['name'], len(records), e.response.status_code), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
                    except:
                        Stat.warning('%s: Failed to upload %d records' % (device['name'], len(records)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
                else:
                    Stat.warning('%s: No accumulated data found. Skipping.' % (device['name']), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            else:
                Stat.warning('%s: Failed to find/create the ESDR feed. Skipping.' % (device['name']), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        else:
            Stat.warning('Failed to find/create the ESDR device for city id %d' % (city_id), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    else:
        Stat.warning('Skipping upload of unknown city id %d' % (city_id), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

In [0]:
def upload_ranking(ranking):
    global esdr, highest_five_ranking_esdr_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not highest_five_ranking_esdr_product:
        highest_five_ranking_esdr_product = get_highest_five_ranking_esdr_product()

    # TODO...
    print("Uploading ranking [%d]" % ranking)

In [0]:
def upload_accumulated():
    global accumulated_cities, accumulated_rankings, uploaded_file_timestamps_db

    refresh_city_info_cache()

    for city_id in sorted(accumulated_cities.keys()):
        upload_city(city_id)
    for ranking in sorted(accumulated_rankings.keys()):
        upload_ranking(ranking)
    for src in sorted(accumulated_file_timestamps):
        uploaded_file_timestamps_db[src] = accumulated_file_timestamps[src]
        Stat.debug('Uploaded %s to ESDR' % (src), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    clear_accumulated()

In [0]:
def is_unmodified(src):
    global uploaded_file_timestamps_db
    return os.path.getmtime(src) == uploaded_file_timestamps_db[src]

In [0]:
def process_all():
    starting_timestamp = time.time()
    clear_accumulated()
    data_files = sorted(glob.glob(AirnowCommon.HIGHEST_FIVE_AQI_DAT_DIRECTORY + '/[0-9]*.dat'))
    Stat.info('Processing %d data files...' % (len(data_files)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

    for src in data_files:
        if len(accumulated_file_timestamps) == NUM_FILES_PER_UPLOAD_BATCH:
            upload_accumulated()
        try:
            if is_unmodified(src):
                continue
        except:
            pass

        process_dat_file(src)
    upload_accumulated()
    ending_timestamp = time.time()
    Stat.up('Done processing %d data files' % (len(data_files)), details='Took %.1f seconds' % (ending_timestamp - starting_timestamp), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME, valid_for_secs=RUN_INTERVAL_SECONDS*1.5)

def process_all_forever():
    while True:
        process_all()
        sleep_until_next_period(RUN_INTERVAL_SECONDS, 1*60)  # start at 1 minutes after the hour


process_all_forever()
